# Домашнее задание 2

## pandas + парсинг

Здесь находится главная страница [Системного блока](https://sysblok.ru)

Соберите корпус новостей (можно спарсить все новости или небольшую часть)

Дедлайн: 31 декабря включительно

In [1]:
# необходимые импорты
import requests as rq
from bs4 import BeautifulSoup
import pandas as pd

### шаг 1

In [2]:
# соберите все ссылки на страницы с новостями
# 1 страница: https://sysblok.ru
# 2 страница: https://sysblok.ru/page/2
# последняя: https://sysblok.ru/page/16

all_page = ['https://sysblok.ru']
for i in range (2,17):
    all_page.append(f"https://sysblok.ru/page/{i}")
print (all_page)

['https://sysblok.ru', 'https://sysblok.ru/page/2', 'https://sysblok.ru/page/3', 'https://sysblok.ru/page/4', 'https://sysblok.ru/page/5', 'https://sysblok.ru/page/6', 'https://sysblok.ru/page/7', 'https://sysblok.ru/page/8', 'https://sysblok.ru/page/9', 'https://sysblok.ru/page/10', 'https://sysblok.ru/page/11', 'https://sysblok.ru/page/12', 'https://sysblok.ru/page/13', 'https://sysblok.ru/page/14', 'https://sysblok.ru/page/15', 'https://sysblok.ru/page/16']


In [41]:
for page in all_page:
    page1 = rq.get(page)
    print(page1.status_code)

In [38]:
for page in all_page:
    soup = BeautifulSoup(page1.text, features="html.parser")
    print(soup.prettify())

### шаг 2

In [42]:
# с каждой страницы соберите ссылки на отдельные новости
# подсказка: родительский тег h2 (проверяйте по нему), нужный нам - a
full_urls = []

for page in all_page:
    for i in soup.find_all('a'):
        if i.parent.name == 'h2':
            full_urls.append(i.get('href'))

full_urls

### шаг 3

работаем с 1 новостью:

нас интересует название, автор, дата публикации, текст (дополнительно можно спарсить тематические категории)

In [6]:
# для парсинга даты: извлеките текст из тега time
url0 = full_urls[0]
page0 = rq.get(url0)

In [39]:
soup0 = BeautifulSoup(page0.text)
print(soup0.prettify())

In [8]:
soup0.find('time').get_text()

'05.10.2018'

In [10]:
# для парсинга заголовка: извлеките текст из тега h1
print(soup0.find_all('h1')[0].get_text())

Робот прикинулся учителем. Никто не заметил


In [43]:
# для парсинга текста: соберите все тексты из тега p и соедините в строку
text_list = soup0.find_all('p')

text = []
for i in text_list:
    
    if i.text:
        text.append(i.text)

final_text = ' '.join(text)
final_text = final_text.replace('\n', '')
final_text

In [14]:
# автор:
for i in soup0.find_all('meta'):
    if i.get('name') == 'author':
        print(i.get("content"))

Татьяна Дубинина


In [15]:
# тематические категории:
for i in soup0.find_all('a'):
    
    if i.get('rel') is not None and i.get('rel')[0] == 'tag':
        print(i.get_text())

образование
общество
робот


### шаг 4

In [16]:
# пишем функцию по сбору информации
def GetNews(url0):
    page0 = rq.get(url0)
    soup0 = BeautifulSoup(page0.text, features="html.parser")
    
    time = soup0.find('time').get_text()
    author = soup0.find_all('meta', {'name' : 'author'})[0].attrs['content']
    
    title = soup0.find_all('h1')[0].get_text()
    
    tags = []
    for i in soup0.find_all('a'):
        if i.get('rel') is not None and i.get('rel')[0] == 'tag':
            tag = i.get_text()
            tags.append(tag)
            
    text_list = soup0.find_all('p')
    text = [i.text for i in text_list if i.text]
    final_text = ' '.join(text)
    final_text = final_text.replace('\n', '')
    return url0, author, time, title, tags, final_text

In [17]:
GetNews(url0)

('https://sysblok.ru/society/robot-prikinulsja-uchitelem-nikto-ne-zametil/',
 'Татьяна Дубинина',
 '05.10.2018',
 'Робот прикинулся учителем. Никто не заметил',
 ['образование', 'общество', 'робот'],
 'Электронный ассистент успешно выдавал себя за живого, переписываясь со студентами и рассылая задания… С днем учителя! Профессор Технологического института Джорджии (Georgia Tech) доверил обязанности своего ассистента боту. Студенты даже не догадались, что общаются не с человеком. «Джилл Уотсон» (Jill Watson) выполняла для Гоэла рутинную работу ассистента: отвечала на вопросы, которые студенты задавали на форуме, рассылала им по электронной почте напоминания о важных событиях… Все происходило настолько «по-человечески», что никому из студентов не пришло в голову , что они переписываются с ботом. Робот Джилл сделана на основе системы IBM Watson. В процессе обучения Джилл исследователи из Georgia Tech предъявили ей более 40 000 постов из обсуждений на образовательной платформе Piazza, а так

In [18]:
from tqdm.notebook import tqdm

In [19]:
# собираем все новости!
# полезно использовать try / except
news = []

for link in tqdm(full_urls):
    try:
        new = GetNews(link)
        news.append(new)
    except:
        print(link)

  0%|          | 0/656 [00:00<?, ?it/s]

In [40]:
news

### шаг 5

In [21]:
# соберите всю собранную информацию в датафрейм


In [ ]:
df = pd.DataFrame(news)

In [31]:
df[4] = df[4].apply(lambda x: ", ".join(x))

In [33]:
df.head()

,0,1,2,3,4,5
0,https://sysblok.ru/society/robot-prikinulsja-u...,Татьяна Дубинина,05.10.2018,Робот прикинулся учителем. Никто не заметил,"образование, общество, робот",Электронный ассистент успешно выдавал себя за ...
1,https://sysblok.ru/society/bolshoj-brat-na-mil...,Инга Картозия,01.10.2018,Большой Брат на миллиард,,Как в век интернета и соцсетей построить оруэл...
2,https://sysblok.ru/arts/koshmarnye-sny-nejrose...,Системный Блокъ,30.09.2018,Кошмарные сны нейросетей,"искусственный интеллект, искусство, нейросети",Как алгоритмы научились видеть ночные кошмары ...
3,https://sysblok.ru/education/poshhady-ne-budet...,Олий Курилов,28.09.2018,Пощады не будет: учителей меняют на роботов,,Преподаешь? Роботы уже готовы лишить тебя рабо...
4,https://sysblok.ru/courses/reguljarnye-vyrazhe...,Мариана Зорькина,27.09.2018,Регулярные выражения. Часть 2,,"Как найти все цифры в учебнике по истории, амп..."


In [34]:
df.columns = ['link', 'author', 'date', 'title', "tags", 'text']

In [35]:
df.head()

,link,author,date,title,tags,text
0,https://sysblok.ru/society/robot-prikinulsja-u...,Татьяна Дубинина,05.10.2018,Робот прикинулся учителем. Никто не заметил,"образование, общество, робот",Электронный ассистент успешно выдавал себя за ...
1,https://sysblok.ru/society/bolshoj-brat-na-mil...,Инга Картозия,01.10.2018,Большой Брат на миллиард,,Как в век интернета и соцсетей построить оруэл...
2,https://sysblok.ru/arts/koshmarnye-sny-nejrose...,Системный Блокъ,30.09.2018,Кошмарные сны нейросетей,"искусственный интеллект, искусство, нейросети",Как алгоритмы научились видеть ночные кошмары ...
3,https://sysblok.ru/education/poshhady-ne-budet...,Олий Курилов,28.09.2018,Пощады не будет: учителей меняют на роботов,,Преподаешь? Роботы уже готовы лишить тебя рабо...
4,https://sysblok.ru/courses/reguljarnye-vyrazhe...,Мариана Зорькина,27.09.2018,Регулярные выражения. Часть 2,,"Как найти все цифры в учебнике по истории, амп..."


Готово!
Загрузите ваш код и собранные новости на GitHub

In [37]:
df.to_excel('sysblok_news.xlsx')